In [ ]:
!pip install -U aicrowd-cli==0.1

In [ ]:
!pip install numpy pandas torch torchvision scikit-learn

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import os
import time
import copy

JAKES_PATH = ""

SINDHUS_PATH = "/content/drive/MyDrive/CS7643-Group-Project-main/Task1/"
#from google.colab import drive
#drive.mount('/content/drive')

# TODO: Select your relative path.
RELATIVE_PATH = JAKES_PATH

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
# Run this to get the data set if you haven't already.
# Create and move data to Task1/data directory.
JAKES_KEY = "8dd3f8e170784e01684fd55ba8c58974"
SINDHUS_KEY = "97685833a2d34a15a1cea9d9e04c5768"
APRILS_KEY = None
MARKS_KEY = None
API_KEY = JAKES_KEY

#!aicrowd login --api-key $API_KEY

In [ ]:
train = np.load(os.path.join(RELATIVE_PATH,'data/train.npy'),allow_pickle=True).item()
#test = np.load(os.path.join(RELATIVE_PATH, 'data/test.npy'),allow_pickle=True).item()
#sample_submission = np.load(os.path.join(RELATIVE_PATH, 'data/sample_submission.npy'),allow_pickle=True).item()

In [ ]:
# Controls how much past and future frames are included.
# 2 implies that every other frame will be included
# Ex num_trajectory_frames = 100 and skip_factor = 2 then
# Training data for curr frame will include data from frames:
# current_frame - skip_factor * num_trajectory_frames, current_frame - (skip_factor * num_trajectory_frames + 2)..., current_frame, ... current_frame + skip_factor,... current_frame + skip_factor * num_trajectory_frames
skip_factor = 2
num_trajectory_frames = 10

train_data_2 = []
labels_data_2 = []
num_train_sequences = len(train['sequences'])
sequence_names = list(train["sequences"].keys())
num_features = len(train['sequences'][sequence_names[0]]["keypoints"][0].flatten())
for seq_id in range(num_train_sequences):

    print(seq_id)
    sequence_key = sequence_names[seq_id]
    single_sequence = train["sequences"][sequence_key]
    num_frames = len(single_sequence['annotations'])
    sequence_data_samples = []
    sequence_annotations = []

    #labels_data.append(single_sequence['annotations'][0])
    for frame in range(num_frames):

        curr_keypoints = single_sequence['keypoints'][frame].flatten()

        # build the past frame data
        past_frames = []
        for i in range(frame - (num_trajectory_frames * skip_factor), frame, skip_factor):

            if i > 0:
                past_frames.append(single_sequence['keypoints'][i].flatten())
            else:
                past_frames.append(np.zeros(num_features))

        # build the future frame data
        future_frames = []
        for i in range(frame + 1, frame + 1 + (num_trajectory_frames * skip_factor), skip_factor):

            if i < num_frames:
                future_frames.append(single_sequence['keypoints'][i].flatten())
            else:
                future_frames.append(np.zeros(num_features))

        data_sample = curr_keypoints

        if len(past_frames) > 0:
            data_sample = np.vstack((np.asarray(past_frames), data_sample))

        if len(future_frames) > 0:
            data_sample = np.vstack((data_sample, np.asarray(future_frames)))

        data_sample = data_sample.T

        # Stack the sequence ID in the first column, and the annotations id in the last column
        data_sample = np.hstack((seq_id * np.ones((num_features, 1)), data_sample, single_sequence['annotations'][frame] * np.ones((num_features, 1))))
        train_data_2.append(data_sample)

train_data_2 = np.asarray(train_data_2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [ ]:
test_data = []
num_test_sequences = len(test['sequences'])
sequence_names = list(test["sequences"].keys())
for seq_id in range(num_test_sequences):

    print(seq_id)
    sequence_key = sequence_names[seq_id]
    single_sequence = test["sequences"][sequence_key]
    num_frames = single_sequence['keypoints'].shape[0]

    #labels_data.append(single_sequence['annotations'][0])
    for frame in range(num_frames):
        curr_keypoints = single_sequence['keypoints'][frame].flatten()

        # build the past frame data
        past_frames = []
        for i in range(frame - (num_trajectory_frames * skip_factor), frame, skip_factor):

            if i > 0:
                past_frames.append(single_sequence['keypoints'][i].flatten())
            else:
                past_frames.append(np.zeros(num_features))

        # build the future frame data
        future_frames = []
        for i in range(frame + 1, frame + 1 + (num_trajectory_frames * skip_factor), skip_factor):

            if i < num_frames:
                future_frames.append(single_sequence['keypoints'][i].flatten())
            else:
                future_frames.append(np.zeros(num_features))

        data_sample = curr_keypoints

        if len(past_frames) > 0:
            data_sample = np.vstack((np.asarray(past_frames), data_sample))

        if len(future_frames) > 0:
            data_sample = np.vstack((data_sample, np.asarray(future_frames)))


        test_data.append(data_sample.T)

In [ ]:
print(len(test_data))

In [ ]:
from sklearn.model_selection import ShuffleSplit

sequence_names = list(train["sequences"].keys())

x = np.arange(len(sequence_names))

rs = ShuffleSplit(test_size=0.10, random_state=42)
train_indices, test_indices = next(rs.split(x))

X_train = train_data_2[np.isin(train_data_2[:,0,0], train_indices)]

# Labels stored in last column
y_train = X_train[:, -1, -1].astype(int)

# Remove the sequence id from the training data (1: up to label column)
X_train = X_train[:,:, 1:-1]

X_val = train_data_2[np.isin(train_data_2[:,0,0], test_indices)]

# Labels stored in last column
y_val = X_val[:, -1, -1].astype(int)

# Remove the sequence id from the training data (1: up to label column)
X_val = X_val[:,:, 1:-1]

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
#https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class MHBDataset(Dataset):
    """Mouse Behavior Dataset"""

    def __init__(self, data, labels, transform=None):
        """
        Args:
            data : numpy array (train/test)
            labels : labels given if train
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        X = self.data[idx].astype(np.float32)
        if self.labels is not None:
          y = self.labels[idx]
        else:
          y = -1
        return X,y

In [ ]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        """
        :param input_dim: input feature dimension
        :param hidden_size: hidden dimension
        :param num_classes: total number of classes
        """
        super(TwoLayerNet, self).__init__()
        #############################################################################
        # TODO: Initialize the TwoLayerNet, use sigmoid activation between layers   #
        #############################################################################
        self.f = nn.Flatten()
        self.layer1 = nn.Linear(input_dim,hidden_size)
        self.layer2 = nn.Linear(hidden_size,hidden_size*2)
        self.layer3 = nn.Linear(hidden_size*2,num_classes)
        #############################################################################
        #                              END OF YOUR CODE                             #
        #############################################################################

    def forward(self, x):
        out = None
        #############################################################################
        # TODO: Implement forward pass of the network                               #
        #############################################################################

        x = self.f(x)
        x = self.layer1(x)
        x = torch.sigmoid(x)
        x = self.layer2(x)
        x = torch.sigmoid(x)
        x = self.layer3(x)
        out = torch.relu(x)
        #############################################################################
        #                              END OF YOUR CODE                             #
        #############################################################################
        return out

In [12]:
#@title
import Models.ConvolutionModel as oneDConvModel

import importlib
importlib.reload(oneDConvModel)

model_2layer =  oneDConvModel.ConvolutionalNet(input_dim = 28, hidden_size = 576, num_classes = 4)

In [16]:
train_dl = MHBDataset(X_train,y_train)
val_dl = MHBDataset(X_val,y_val)
dataloaders = {
    'train': DataLoader(train_dl, batch_size=32,shuffle=True, num_workers=0),
    'val': DataLoader(val_dl, batch_size=32,shuffle=True, num_workers=0)
}
dataset_sizes = {
    'train':len(y_train),
    'val':len(y_val)
}
history = {
    'epoch':[],
    'train_loss':[],
    'val_loss':[],
    'train_acc':[],
    'val_acc':[],
    'best_acc':0,
    'best_epoch':0
}


In [19]:
import CommonUtils.Utilites as utils
import importlib
importlib.reload(utils)

model_2layer = utils.train_model(model_2layer,dataloaders,num_epochs=40)
torch.save(model_2layer.state_dict(),os.path.join(RELATIVE_PATH,'model_weights.pth'))

Epoch 0/39
----------
train Loss: 0.2622 Acc: 0.9054
val Loss: 0.5791 Acc: 0.8008

Epoch 1/39
----------
train Loss: 0.2570 Acc: 0.9075
val Loss: 0.5341 Acc: 0.8189

Epoch 2/39
----------
train Loss: 0.2527 Acc: 0.9092
val Loss: 0.5312 Acc: 0.8081

Epoch 3/39
----------
train Loss: 0.2487 Acc: 0.9106
val Loss: 0.5122 Acc: 0.8265

Epoch 4/39
----------
train Loss: 0.2449 Acc: 0.9123
val Loss: 0.5851 Acc: 0.8132

Epoch 5/39
----------
train Loss: 0.2421 Acc: 0.9135
val Loss: 0.6140 Acc: 0.7884

Epoch 6/39
----------
train Loss: 0.2389 Acc: 0.9150
val Loss: 0.5897 Acc: 0.7946

Epoch 7/39
----------
train Loss: 0.2373 Acc: 0.9157
val Loss: 0.5672 Acc: 0.7976

Epoch 8/39
----------
train Loss: 0.2342 Acc: 0.9169
val Loss: 0.5896 Acc: 0.7965

Epoch 9/39
----------
train Loss: 0.2324 Acc: 0.9179
val Loss: 0.5822 Acc: 0.7947

Epoch 10/39
----------
train Loss: 0.2301 Acc: 0.9185
val Loss: 0.5888 Acc: 0.8053

Epoch 11/39
----------
train Loss: 0.2281 Acc: 0.9196
val Loss: 0.6693 Acc: 0.7839

Ep

In [18]:
PATH = os.path.join(RELATIVE_PATH,'model_weights.pth')
model = model_2layer.to(device)
model.load_state_dict(torch.load(PATH))
model.eval()

ConvolutionalNet(
  (layer1): Conv1d(28, 64, kernel_size=(3,), stride=(1,))
  (layer1_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer3): Flatten(start_dim=1, end_dim=-1)
  (layer4): Linear(in_features=576, out_features=288, bias=True)
  (layer4_norm): BatchNorm1d(288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer5): Linear(in_features=288, out_features=4, bias=True)
  (layer5_norm): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
output = []
test_dl = MHBDataset(test_data,None)
testDL = DataLoader(test_dl, batch_size=256,shuffle=False, num_workers=0)
for inputs, _ in testDL:
  inputs = inputs.to(device)
  outputs = model(inputs)
  _, preds = torch.max(outputs, 1)
  out = preds.cpu().numpy().flatten()
  output.extend(out)

In [ ]:
output = np.array(output)
submission = {}
test = np.load(os.path.join(RELATIVE_PATH,'data/test.npy'),allow_pickle=True).item()
i = 0
for sequence_id, sequence in test["sequences"].items():
  keypoint_sequence = sequence['keypoints']
  submission[sequence_id] = np.array(output[i:i+len(keypoint_sequence)])
  i += len(keypoint_sequence)

In [ ]:
def validate_submission(submission, sample_submission):
    if not isinstance(submission, dict):
      print("Submission should be dict")
      return False

    if not submission.keys() == sample_submission.keys():
      print("Submission keys don't match")
      return False

    for key in submission:
      sv = submission[key]
      ssv = sample_submission[key]
      if not len(sv) == len(ssv):
        print(f"Submission lengths of {key} doesn't match")
        return False

    for key, sv in submission.items():
      if not all(isinstance(x, (np.int32, np.int64, int)) for x in list(sv)):
        print(f"Submission of {key} is not all integers")
        return False

    print("All tests passed")
    return True

In [ ]:
print(submission)

In [ ]:
validate_submission(submission, sample_submission)

In [ ]:
np.save("submission.npy", submission)

In [ ]:
!pip install -U aicrowd-cli==0.1
!aicrowd login --api-key $API_KEY
!aicrowd submission create -c mabe-task-1-classical-classification -f submission.npy